In [ ]:
#
# All code cells are hidden in the output by default
#

# Parameter cell. Wilk be replaced by export_backtest_report()
parameters = {}

In [ ]:
#
# Setting up
#


# Loads strategy trades and universe as passed over
# by the host Python system as temp files
from tradeexecutor.backtest.tearsheet import BacktestReporter
reporter = BacktestReporter.setup_report(parameters)
state = reporter.get_state()
universe = reporter.get_universe()

In [ ]:
## Metric calculations

# Calculate different chart data and metrics.
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns

curve = calculate_equity_curve(state)
returns = calculate_returns(curve)
first_trade, last_trade = state.portfolio.get_first_and_last_executed_trade()
start_at = state.backtest_data.start_at
end_at = state.backtest_data.end_at
trades = list(state.portfolio.get_all_trades())
name = state.name

# Preface

Information about the executed backtest.

In [ ]:
import pandas as pd

data = {
    "Name": name,
    "Run at": state.created_at,
    "Backtesting period start": start_at,
    "Backtesting period end": end_at,
    "Trades": len(trades),
}

# Display dictionary as a pretty table output
# display(pd.DataFrame(data.items()).style.hide(axis="columns").hide(axis="index"))
display(pd.DataFrame(data.values(), index=data.keys()).style.hide(axis="columns"))

# Equity curve

Equity curve, maximum drawdown and daily profit.

In [ ]:
from tradeexecutor.visual.equity_curve import visualise_equity_curve

visualise_equity_curve(returns)

# Performance metrics

Portfolio key performance metrics.

In [ ]:
import pandas as pd
from tradeexecutor.analysis.advanced_metrics import visualise_advanced_metrics, AdvancedMetricsMode

metrics = visualise_advanced_metrics(returns, mode=AdvancedMetricsMode.full)

with pd.option_context("display.max_row", None):
    display(metrics)

# Trading metrics

Calculate key trading metrics.



In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)
summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

# Trading positions

Display trading positions over time.


In [ ]:

from tradeexecutor.visual.single_pair import visualise_single_pair_positions_with_duration_and_slippage

if universe.universe.pairs.get_count() == 1:
    pair_id = int(universe.get_single_pair().internal_id)
    candles = universe.universe.candles.get_candles_by_pair(pair_id)

    fig = visualise_single_pair_positions_with_duration_and_slippage(
        state,
        candles,
        start_at=start_at,
        end_at=end_at,
        pair_id=pair_id,
    )

    fig.show()
else:
    print("")

# Benchmark

Compare the strategy results against

- Buy and hold benchmark
- All-cash benchmark

In [ ]:
if universe.universe.pairs.get_count() == 1:
    from tradeexecutor.visual.benchmark import visualise_benchmark

    traded_pair = universe.universe.pairs.get_single()

    fig = visualise_benchmark(
        name,
        portfolio_statistics=state.stats.portfolio,
        all_cash=state.portfolio.get_initial_deposit(),
        buy_and_hold_asset_name=traded_pair.base_token_symbol,
        buy_and_hold_price_series=universe.universe.candles.get_single_pair_data()["close"],
        start_at=start_at,
        end_at=end_at,
        height=800
    )

    fig.show()
else:
    print("Benchmark part for a multipair strategy not yet implemented")

# Monthly returns

Returns by a month.


In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_over_time

visualise_returns_over_time(returns)

# Periodic return distribution

Show performance variations for different timeframes.

In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_distribution

visualise_returns_distribution(returns)

# Trade details

Show details of every position open and close.

In [ ]:
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
        universe.universe.exchanges,
        universe.universe.pairs,
        timeline)

# Do not truncate the row output
with pd.option_context("display.max_row", None):
    display(apply_styles(expanded_timeline))